In [ ]:
"""
Train the model using Yolov8 and Roboflow

Reference: https://blog.roboflow.com/how-to-train-yolov8-on-a-custom-dataset/
"""

In [1]:
# if not installed run: pip install ultralytics
import ultralytics
print(ultralytics.checks())

Ultralytics YOLOv8.0.232 🚀 Python-3.10.12 torch-2.2.0+cu121 CPU (AMD Ryzen 9 6900HS with Radeon Graphics)
Setup complete ✅ (16 CPUs, 14.9 GB RAM, 77.9/97.5 GB disk)
None


In [5]:
# if not installed run: pip install roboflow
# if you have a weird 'appengine' error run this: pip install urllib3==2.0.7
# other errors, run through chat gpt
# this was copied from exporting the dataset on roboflow with 'yolov8'

from roboflow import Roboflow

rf = Roboflow(api_key="tG58USi1UKOd6l1rkJEw") # Private Key for Team
project = rf.workspace("arcturus-gx59n").project("buoy-labeling-full")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.232, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to Buoy-Labeling-Full-1 in yolov8:: 100%|██████████| 2006/2006 [00:00<00:00, 11454.81it/s]


In [ ]:
"""Reference: @misc{
roboboat-comp-2024_dataset,
title = { RoboBoat Comp 2024 Dataset },
type = { Open Source Dataset },
author = { Cornell AutoBoat },
howpublished = { '\'url{ https://universe.roboflow.com/cornell-autoboat-fz9yv/roboboat-comp-2024 } },
url = { https://universe.roboflow.com/cornell-autoboat-fz9yv/roboboat-comp-2024 },
journal = { Roboflow Universe },
publisher = { Roboflow },
year = { 2024 },
month = { feb },
note = { visited on 2024-03-17 },
}"""

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="UEdSpNWdrrkxTs2Uz6JD")
project = rf.workspace("cornell-autoboat-fz9yv").project("roboboat-comp-2024")
version = project.version(4)
dataset = version.download("yolov8")


In [ ]:
# Run the training of the model with the correct location to the data.yaml and custom settings (could take a while)
# This assumes data folder is in perception folder and this document is in perception folder
data_folder = 'Buoy-Labeling-Full-1'

imgsz = 640

model= 'yolov8n.pt' # the other models are too big

# ! symbol lets you run command line interface commands (terminal commands) 
# Run this in command line if you get errors
# replace data.location with the name of the file folder
# WARNING: This will take a while and consume a lot of computer resources
!yolo task=detect \
    mode=train \
    model=yolov8s.pt \
    data={data.location}/data.yaml \
    epochs=25 \
    imgsz=640 \
    plots=True


In [ ]:
# validation of training results

!yolo task=detect \
    mode=val \
    model={HOME}/runs/detect/train/weights/best.pt \
    data={dataset.location}/data.yaml

In [12]:
# predict using model

!yolo task=detect \
    mode=predict \
    model={HOME}/runs/detect/train/weights/best.pt \
    conf=0.25 \
    source={dataset.location}/test/images

/home/arthurdls/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
Traceback (most recent call last):
  File "/home/arthurdls/.local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
  File "/home/arthurdls/.local/lib/python3.10/site-packages/ultralytics/cfg/__init__.py", line 423, in entrypoint
    model = YOLO(model, task=task)
  File "/home/arthurdls/.local/lib/python3.10/site-packages/ultralytics/engine/model.py", line 94, in __init__
    self._l

In [2]:

from ultralytics import YOLO
model = YOLO("runs/detect/train_roboboat/weights/best.pt")

# results = model.predict(source="RoboBoat-Comp-2024-4/test/images")
results = model(["arcturus_test_image_1.jpg",
                 "arcturus_test_image_2.jpg",
                 "arcturus_test_image_3.jpg",
                 "arcturus_test_image_4.jpg"])
for i, result in enumerate(results):
    print(f"res = {result}")
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    result.save(filename=f'result_{i}.jpg')  # specify filename to save to disk


0: 480x640 1 red-buoy, 1 yellow-buoy, 1 yellow-duck, 26.3ms
1: 480x640 2 green-buoys, 1 red-buoy, 1 yellow-buoy, 26.3ms
2: 480x640 2 green-buoys, 1 red-buoy, 1 yellow-buoy, 1 yellow-duck, 26.3ms
3: 480x640 3 green-buoys, 1 red-buoy, 1 yellow-buoy, 26.3ms
Speed: 3.5ms preprocess, 26.3ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)
res = ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'black-buoy', 1: 'blue-buoy', 2: 'blue-cross', 3: 'green-buoy', 4: 'green-column-buoy', 5: 'red-buoy', 6: 'red-column-buoy', 7: 'red-dot', 8: 'yellow-buoy', 9: 'yellow-duck'}
obb: None
orig_img: array([[[ 35,   4, 213],
        [ 34,   3, 212],
        [ 37,   6, 215],
        ...,
        [ 62,  81, 102],
        [ 42,  61,  82],
        [ 60,  79, 100]],

       [[ 38,   7, 216],
        [ 36,   5, 214],
        [ 38,   7, 216],
        ...,
        [ 64,  83, 104],
        [ 60,  79, 100